# Assignment 2: Data Cleaning and EDA
## Critical Role Campaign 2: Dice Roll Analysis
Nicole Gunther Guerrero

**Dataset:** All recorded dice rolls from *Critical Role* Campaign 2  
**Source:** Fan-compiled data from CritRole Stats https://www.critrolestats.com/stats-wm 

### Dataset structure

- Episode	
- Time	
- Character	
- Type of Roll	
- Total Value	
- Natural Value	
- Crit?	
- Damage Dealt	
- "# Kills"	
- Notes

#### What is Critical Role?

*Critical Role* is a web series where professional voice actors play Dungeons & Dragons. Campaign 2 follows the adventures of a party known as the **Mighty Nein** as they journey across the continent of Wildemount. The show has a massive fanbase and streams weekly on Twitch and YouTube.

In Dungeons & Dragons, gameplay depends on dice (a twenty sided dice in this case), rolls that determine success or failure in combat, social interaction, and skill-based challenges. Every action, from persuasion to smiting a demon, starts with a roll.

### Why Analyse Dice Rolls?

This project explores the storytelling patterns, skills, battles, and player behaviours behind the scenes of *Critical Role*'s second campaign. We'll investigate:
- Individual player luck (critical hits/fails) - when the dice lands on a 20 or 1.
- Roll type distributions
- Episode pacing and roll frequency
- Combat vs. Role-Play (skill checks)

The goal is to turn raw dice data into a story: who took the biggest risks? Who had the worst luck streaks? And is the dice god truly chaotic?

***
## Data Loading
To start we will import the necesary libraries as well as the dataset.
- pandas -> data manipulation and analysis.
- re -> search, extract, manipulate text patterns (for data cleaning).
- plotly.express and plotly.graph_objects -> interactive visualisations.
- make_subplots from plotly.subplots -> combine multiple plots into one figure.
- seaborn -> better looking plots and more information in them.

And get our first view on the structure and general information.

In [31]:
import pandas as pd
import re
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

# Load dataset
df = pd.read_csv("CritRole_M9_allRolls.csv")

# Show the first 5 rows to get a sense of the data
df.head()


,Episode,Time,Character,Type of Roll,Total Value,Natural Value,Crit?,Damage Dealt,# Kills,Notes
0,C2E001,0:35:40,Fjord,Perception,Nat20,20,NaN,NaN,NaN,NaN
1,C2E001,0:39:22,Jester,Perception,22,19,NaN,NaN,NaN,NaN
2,C2E001,0:39:30,Beau,Perception,19,16,NaN,NaN,NaN,NaN
3,C2E001,0:43:04,Beau,Insight,13,8,NaN,NaN,NaN,NaN
4,C2E001,0:50:37,Molly,Insight,16,11,NaN,NaN,NaN,NaN


In [32]:
# Lets check the structure of the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15364 entries, 0 to 15363
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Episode        15363 non-null  object
 1   Time           15364 non-null  object
 2   Character      15364 non-null  object
 3   Type of Roll   15364 non-null  object
 4   Total Value    15358 non-null  object
 5   Natural Value  12553 non-null  object
 6   Crit?          89 non-null     object
 7   Damage Dealt   2518 non-null   object
 8   # Kills        338 non-null    object
 9   Notes          7510 non-null   object
dtypes: object(10)
memory usage: 1.2+ MB
None


In [33]:
df.isnull().sum()

Episode              1
Time                 0
Character            0
Type of Roll         0
Total Value          6
Natural Value     2811
Crit?            15275
Damage Dealt     12846
# Kills          15026
Notes             7854
dtype: int64

Check for duplicates and deal with them if found.

In [ ]:
# Check for duplicates 
df.duplicated().sum()


np.int64(36)

In [35]:
# Drop duplicates if any
df.drop_duplicates(inplace=True)

In [36]:
# Check for duplicates again
df.duplicated().sum()

np.int64(0)

***
## Data Cleaning

### Processing Episode Information

The `Episode` column contains strings like `C2E001`, which indicate the episode number in Campaign 2. For analysis and plotting purposes, it's helpful to extract the episode number as an integer so we can sort episodes chronologically.

In this step, we will:
- Create a new column called `Episode_Num` that contains the numerical episode value
- Convert it to an integer for proper ordering and plotting


In [37]:
# Extract the numeric part of the episode code and convert it to an integer
df["Episode_Num"] = df["Episode"].str.extract(r"C2E(\d+)", expand=False)
df["Episode_Num"] = pd.to_numeric(df["Episode_Num"], errors="coerce")

# Quick check: show first few unique episode numbers
print("Unique episode numbers found:", sorted(df["Episode_Num"].dropna().unique())[:10])

# Turn to integer
df["Episode_Num"] = df["Episode_Num"].astype("Int64")


Unique episode numbers found: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0)]


### Standardising Main Character Names

To avoid duplicate entries caused by the use of player names or aliases, we are mapping all character references to their consistent campaign names. This ensures accurate analysis for each member of the Mighty Nein.

In [38]:
# Display all unique character names, sorted alphabetically
unique_names = sorted(df["Character"].dropna().unique())
for name in unique_names:
    print(f"- {name}")

- Balleater
- Beau
- Beetles
- Caduceus
- Caleb
- Cali
- Clarabelle
- Duchess
- Fjord
- Frumpkin
- Jannik
- Jester
- Keg
- Lightning Spirit Liam
- Lightning Spirit Taliesin
- Lightning Spirit Travis
- Molly
- Nila
- Nott
- Nugget
- Reani
- Shakäste
- Sprinkle
- Spurt
- Storm Spirit Laura
- Storm Spirit Marisha
- Storm Spirit Sam
- Summoned Creature
- Travis
- Twiggy
- Veth
- Willi
- Willie
- Yarnball
- Yasha


In [39]:
# Map of all alternate names to core character names
character_fixes = {
    "Marisha": "Beau",
    "Travis": "Fjord",
    "Laura": "Jester",
    "Sam": "Veth",
    "Nott": "Veth",
    "Liam": "Caleb",
    "Ashley": "Yasha",
    "Taliesin": "Caduceus",
    "Mollymauk": "Molly",
}

# Apply the corrections
df["Character"] = df["Character"].replace(character_fixes)


# Check results again
unique_names = sorted(df["Character"].dropna().unique())
for name in unique_names:
    print(f"- {name}")


- Balleater
- Beau
- Beetles
- Caduceus
- Caleb
- Cali
- Clarabelle
- Duchess
- Fjord
- Frumpkin
- Jannik
- Jester
- Keg
- Lightning Spirit Liam
- Lightning Spirit Taliesin
- Lightning Spirit Travis
- Molly
- Nila
- Nugget
- Reani
- Shakäste
- Sprinkle
- Spurt
- Storm Spirit Laura
- Storm Spirit Marisha
- Storm Spirit Sam
- Summoned Creature
- Twiggy
- Veth
- Willi
- Willie
- Yarnball
- Yasha


### Categorising Characters

To simplify analysis, we are grouping characters into three categories:
- **Main cast**: Cleaned and standardised earlier
- **Guest characters**: Played by guest players across various arcs
- **NPCs**: All other characters (typically controlled by the Dungeon Master)

We will identify known guest characters and assign all remaining entries to "NPC".


In [40]:
# Define known guest characters
guest_names = [
    "Reani", "Twiggy", "Keg", "Cali", "Shakäste", "Nila", "Spurt"
]

# Assign guests
df["Character"] = df["Character"].replace({name: "Guest" for name in guest_names})

# Everything else not in main cast or guests becomes NPC
main_cast = [
    "Beau", "Fjord", "Jester", "Veth", "Caleb", "Yasha", "Caduceus", "Molly", "Kingsley"
]

df["Character"] = df["Character"].apply(lambda x: "NPC" if x not in main_cast + ["Guest"] else x)

# Preview updated unique characters
print(sorted(df["Character"].unique()))


['Beau', 'Caduceus', 'Caleb', 'Fjord', 'Guest', 'Jester', 'Molly', 'NPC', 'Veth', 'Yasha']


### Cleaning the "Type of Roll" Column

The `Type of Roll` column contains text describing what kind of dice roll was made (e.g., Perception, Attack, Insight, etc.). However, these values may have inconsistent formatting or naming conventions, making analysis difficult.

In this step, we will:
- Remove leading/trailing whitespace
- Standardise text casing
- Optionally group or rename similar roll types to reduce redundancy


In [41]:
# Remove extra whitespace and capitalise consistently
df["Type of Roll"] = df["Type of Roll"].str.strip().str.title()

# Show the most common roll types to inspect
roll_type_counts = df["Type of Roll"].value_counts()
print("Top 20 Roll Types:")
print(roll_type_counts.head(20))


Top 20 Roll Types:
Type of Roll
Attack               2501
Damage               2111
Perception           1842
Stealth              1112
Initiative            853
Investigation         768
Spell Attack          710
Constitution Save     625
Wisdom Save           526
Dexterity Save        378
Persuasion            306
Athletics             298
Healing               288
Other                 283
Strength              275
Insight               271
Deception             227
Acrobatics            201
Intelligence          165
History               137
Name: count, dtype: int64


### Categorising Roll Types

To simplify analysis, we will group similar roll types into broader categories such as:
- **Skill Check** (e.g., Perception, Stealth, Insight)
- **Attack** (e.g., Melee Attack, Ranged Attack)
- **Saving Throw** (e.g., Dexterity Save, Wisdom Save)
- **Spellcasting** (e.g., Spell Attack, Concentration Check)
- **Other** (miscellaneous or custom rolls)

This step allows us to analyse general player behaviour patterns without being overwhelmed by roll-specific noise.


In [42]:
# Define keyword-based mappings to general categories
def categorize_roll_type(roll):
    if pd.isna(roll):
        return "Other"
    roll = roll.lower()
    if any(skill in roll for skill in ["check", "perception", "insight", "stealth", "investigation", "persuasion", "intimidation", "deception", "acrobatics", "athletics", "survival", "history", "nature", "arcana", "medicine", "religion", "animal handling"]):
        return "Skill Check"
    elif "attack" in roll or "strike" in roll:
        return "Attack"
    elif "save" in roll or "saving throw" in roll:
        return "Saving Throw"
    elif "spell" in roll or "concentration" in roll:
        return "Spellcasting"
    else:
        return "Other"

# Apply the categorisation
df["Roll Category"] = df["Type of Roll"].apply(categorize_roll_type)

# Show counts of each category
category_counts = df["Roll Category"].value_counts()
print("Roll Category Distribution:")
print(category_counts)


Roll Category Distribution:
Roll Category
Skill Check     5754
Other           4674
Attack          3211
Saving Throw    1687
Spellcasting       2
Name: count, dtype: int64


In [43]:
df.head()

,Episode,Time,Character,Type of Roll,Total Value,Natural Value,Crit?,Damage Dealt,# Kills,Notes,Episode_Num,Roll Category
0,C2E001,0:35:40,Fjord,Perception,Nat20,20,NaN,NaN,NaN,NaN,1,Skill Check
1,C2E001,0:39:22,Jester,Perception,22,19,NaN,NaN,NaN,NaN,1,Skill Check
2,C2E001,0:39:30,Beau,Perception,19,16,NaN,NaN,NaN,NaN,1,Skill Check
3,C2E001,0:43:04,Beau,Insight,13,8,NaN,NaN,NaN,NaN,1,Skill Check
4,C2E001,0:50:37,Molly,Insight,16,11,NaN,NaN,NaN,NaN,1,Skill Check


### Cleaning Roll Values

The columns `Total Value` and `Natural Value` represent the results of each dice roll. However, both columns are currently stored as text (strings), which prevents us from performing numerical analysis.

In this step, we will:
- Convert both columns to numeric data types
- Handle any non-numeric values
- Check for any remaining missing or invalid entries


In [44]:
# Convert 'Total Value' and 'Natural Value' to numeric
# Errors will be turned into NaNs (e.g., empty cells or malformed entries)

df["Total Value"] = pd.to_numeric(df["Total Value"], errors="coerce") #  tries to convert to number. If it fails, it replaces with NaN instead of crashing.
df["Natural Value"] = pd.to_numeric(df["Natural Value"], errors="coerce")

# Check how many values were converted to NaN (missing or invalid)
missing_total = df["Total Value"].isna().sum()
missing_natural = df["Natural Value"].isna().sum()

# Turn to integer
df["Total Value"] = df["Total Value"].astype("Int64")
df["Natural Value"] = df["Natural Value"].astype("Int64")

print(f"Missing or invalid 'Total Value' entries: {missing_total}")
print(f"Missing or invalid 'Natural Value' entries: {missing_natural}")


Missing or invalid 'Total Value' entries: 3113
Missing or invalid 'Natural Value' entries: 5043


**Note on Missing Values**

Since this dataset was compiled from a live-streamed roleplaying game, some dice rolls were not verbally stated by the players or were obscured by overlapping dialogue or other event. As a result, several entries in the `Natural Value` and `Total Value` columns are missing or incomplete. This is expected in a community-compiled datasets and should be taken into account during analysis.


### Identifying Natural 1s and Natural 20s

In Dungeons & Dragons, a "natural 1" (rolling a 1 on a 20-sided die) is typically a critical failure, while a "natural 20" is a critical success. These extreme rolls are often the most memorable and impactful moments in the game.

To highlight these moments, we will:
- Create a new column `Is_Nat1` that flags natural 1s
- Create a new column `Is_Nat20` that flags natural 20s
- These will be useful for visualisations and luck-based metrics


In [45]:
# Create boolean columns for natural 1s and natural 20s
df["Is_Nat1"] = df["Natural Value"] == 1
df["Is_Nat20"] = df["Natural Value"] == 20

# Quick summary to see how many of each
nat1_count = df["Is_Nat1"].sum()
nat20_count = df["Is_Nat20"].sum()

print(f"Total Natural 1s: {nat1_count}")
print(f"Total Natural 20s: {nat20_count}")


Total Natural 1s: 532
Total Natural 20s: 647


In [46]:
df.head()

,Episode,Time,Character,Type of Roll,Total Value,Natural Value,Crit?,Damage Dealt,# Kills,Notes,Episode_Num,Roll Category,Is_Nat1,Is_Nat20
0,C2E001,0:35:40,Fjord,Perception,<NA>,20,NaN,NaN,NaN,NaN,1,Skill Check,False,True
1,C2E001,0:39:22,Jester,Perception,22,19,NaN,NaN,NaN,NaN,1,Skill Check,False,False
2,C2E001,0:39:30,Beau,Perception,19,16,NaN,NaN,NaN,NaN,1,Skill Check,False,False
3,C2E001,0:43:04,Beau,Insight,13,8,NaN,NaN,NaN,NaN,1,Skill Check,False,False
4,C2E001,0:50:37,Molly,Insight,16,11,NaN,NaN,NaN,NaN,1,Skill Check,False,False


### Dropping the "Crit?" Column

The `Crit?` column has very limited data and overlaps with information already present in the `Natural Value` column. Since it only contains a handful of entries, we will drop it to clean up the dataset and avoid redundancy.


In [47]:
# Drop the "Crit?" column
df.drop(columns=["Crit?"], inplace=True)

# Confirm it's gone
print("Remaining columns:", df.columns.tolist())

Remaining columns: ['Episode', 'Time', 'Character', 'Type of Roll', 'Total Value', 'Natural Value', 'Damage Dealt', '# Kills', 'Notes', 'Episode_Num', 'Roll Category', 'Is_Nat1', 'Is_Nat20']


### Extracting Numeric Values from "# Kills"

The `# Kills` column contains information about the number of kills associated with a roll, but the values are inconsistently formatted. To use this data in analysis, we will:
- Extract numeric digits from the column
- Convert the result to integers
- Replace missing values with 0


In [48]:
# Use regex to extract digits from the '# Kills' column
df["Kills"] = df["# Kills"].str.extract(r"(\d+)", expand=False)
df["Kills"] = pd.to_numeric(df["Kills"], errors="coerce").fillna(0).astype(int)

# Drop the original messy column
df.drop(columns=["# Kills"], inplace=True)

# Quick check
print(df["Kills"].value_counts().sort_index())

Kills
0    15003
1      309
2       11
3        2
4        2
6        1
Name: count, dtype: int64


### Cleaning the "Damage Dealt" Column

The `Damage Dealt` column contains numerical and text-based entries describing the amount of damage a character dealt during combat. For analysis, we will:
- Extract the first numeric value from the string (ignoring text and extra values)
- Convert it to a int
- Replace missing or invalid entries with 0


In [49]:
# Extract the first number from the 'Damage Dealt' column using regex
df["Damage"] = df["Damage Dealt"].str.extract(r"(\d+)", expand=False)
df["Damage"] = pd.to_numeric(df["Damage"], errors="coerce").fillna(0).astype(int)

# Drop the original messy column
df.drop(columns=["Damage Dealt"], inplace=True)

# Quick check: summary stats
print(df["Damage"].describe())


count    15328.000000
mean         2.483364
std          7.436470
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        172.000000
Name: Damage, dtype: float64


### Reordering Columns for Clarity

To improve readability and usability, we will reorder the columns in a logical sequence:
- Episode information
- Character and roll details
- Dice outcomes
- Consequences of the roll
- Notes for additional context


In [50]:
# Define preferred column order
column_order = [
    "Episode", "Episode_Num", "Time",
    "Character", "Type of Roll", "Roll Category",
    "Natural Value", "Total Value", "Is_Nat1", "Is_Nat20",
    "Damage", "Kills",
    "Notes"
]

# Reorder the DataFrame
df = df[column_order]

# Quick preview
df.head()


,Episode,Episode_Num,Time,Character,Type of Roll,Roll Category,Natural Value,Total Value,Is_Nat1,Is_Nat20,Damage,Kills,Notes
0,C2E001,1,0:35:40,Fjord,Perception,Skill Check,20,<NA>,False,True,0,0,NaN
1,C2E001,1,0:39:22,Jester,Perception,Skill Check,19,22,False,False,0,0,NaN
2,C2E001,1,0:39:30,Beau,Perception,Skill Check,16,19,False,False,0,0,NaN
3,C2E001,1,0:43:04,Beau,Insight,Skill Check,8,13,False,False,0,0,NaN
4,C2E001,1,0:50:37,Molly,Insight,Skill Check,11,16,False,False,0,0,NaN


### Exporting the Cleaned Dataset

Now that the data is cleaned and structured, we will export it to a CSV file. This version will be used later to build the interactive dashboard in Streamlit.


In [59]:
# Export the cleaned dataframe to a CSV file
cleaned_file_path = "critrole_c2_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data exported as: {cleaned_file_path}")


Cleaned data exported as: critrole_c2_cleaned.csv


***
## Visualizations

### Total Rolls by Character

To begin our analysis, we examine how many dice rolls each character made during Campaign 2. This basic frequency count helps us understand overall player activity, spotlighting which characters were most involved in combat, skill checks, or magical chaos.


In [52]:
# Count total rolls per character
character_counts = df["Character"].value_counts().reset_index()
character_counts.columns = ["Character", "Total Rolls"]
character_counts = character_counts.sort_values("Total Rolls", ascending=True)

# Create a horizontal bar chart
fig = px.bar(
    character_counts,
    x="Total Rolls",
    y="Character",
    orientation="h",
    title="Total Rolls per Character (Campaign 2)",
    text="Total Rolls",
    color="Total Rolls",
    color_continuous_scale="Magma"
)

fig.update_layout(
    xaxis_title="Total Rolls",
    yaxis_title="Character",
    title_x=0.5,
    height=500,
    plot_bgcolor="#fdf6f0"
)

fig.update_traces(textposition="outside")

fig.show()


#### Insight: Camaign Dynamic

This graph provides great insight into the dynamic of the cmpaign. We can see that Beau made the most rolls out of all the characters, which makes sense, as she is playing a monk and its known that that class offers more actions in combat. So the next course of action would be to investigate the roll type by character to prove this hypothesis.
***

### Roll Type Breakdown by Character (Interactive)

This interactive chart allows us to explore the breakdown of roll types for each character. By selecting a character from the dropdown menu, we can observe which kinds of actions they performed most, whether it's skill checks, attacks, spellcasting, or saving throws.


In [53]:
# Get the list of unique characters (optional: filter to main characters only)
unique_characters = sorted(df["Character"].unique())

# Create the figure and add one pie trace per character
fig = go.Figure()

for i, char in enumerate(unique_characters):
    char_data = df[df["Character"] == char]["Roll Category"].value_counts().reset_index()
    char_data.columns = ["Roll Category", "Count"]
    
    fig.add_trace(go.Pie(
        labels=char_data["Roll Category"],
        values=char_data["Count"],
        name=char,
        hole=0.4,
        visible=(i == 0),
        textinfo="percent+label"
    ))

# Create the dropdown buttons
dropdown_buttons = [
    dict(label=char,
         method="update",
         args=[
             {"visible": [i == j for j in range(len(unique_characters))]},
             {"title": f"Roll Type Breakdown: {char}"}
         ])
    for i, char in enumerate(unique_characters)
]

# Update the layout with dropdown
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_buttons,
            x=0.5,
            xanchor="center",
            y=1.2
        )
    ],
    title=f"Roll Type Breakdown: {unique_characters[0]}",
    height=600,
    showlegend=True
)

fig.show()

#### Insight: How Each Character Rolled

This chart shows the proportion of roll types made by each character, giving insight into their overall playstyle and in-game role:

- Characters like **Beau** show a balance between **attack rolls** and **skill checks**, reflecting her monk class and investigation-heavy moments.
- **Caduceus** and **Jester** have higher proportions of **"other" rolls**, tied to their frequent use of magical abilities.
- **Fjord** and **Veth** display a balance between **attack rolls** and **saving throws**, as part of their combat-forward styles.
- **Yasha** has the most balanced roll types, and this can be due to her absence on several episodes. 

Overall, this chart helps visualise how each character interacted with the game's mechanics—not just how often they rolled, but *what they were doing* when they did.
***

### Nat 1 vs Nat 20 by character

This chart compares the number of critical failures (natural 1s) and critical successes (natural 20s) for each character. These extreme outcomes often create dramatic and memorable moments during gameplay. By visualising them side by side, we can explore which characters were luckiest or unluckiest, over the course of Campaign 2.

In [54]:
# Filter only main characters
main_characters = ["Beau", "Fjord", "Jester", "Veth", "Caleb", "Yasha", "Caduceus", "Molly", "Kingsley"]
filtered_df = df[df["Character"].isin(main_characters)]

# Count Nat 1s and Nat 20s per character
nat_counts = filtered_df.groupby("Character")[["Is_Nat1", "Is_Nat20"]].sum().reset_index()
nat_counts = pd.melt(nat_counts, id_vars="Character", value_vars=["Is_Nat1", "Is_Nat20"],
                     var_name="Roll Type", value_name="Count")

# Rename roll types for display
nat_counts["Roll Type"] = nat_counts["Roll Type"].map({"Is_Nat1": "Nat 1", "Is_Nat20": "Nat 20"})

# Create grouped bar chart
fig = px.bar(
    nat_counts,
    x="Character",
    y="Count",
    color="Roll Type",
    barmode="group",
    title="🎲 Natural 1s vs Natural 20s by Character",
    text="Count",
    color_discrete_map={"Nat 1": "#FF6666", "Nat 20": "#66FF99"}
)

fig.update_layout(
    xaxis_title="Character",
    yaxis_title="Count",
    height=500
)

fig.show()


#### Insight: Luckiest and Unluckiest Members of the Mighty Nein

This chart highlights some interesting trends in dice fortune:

- Characters like **Beau** and **Fjord** rolled a large number of natural 20s, suggesting frequent moments of peak success, perfect for a monk and warlock.
- **Jester** and **Veth**, on the other hand, experienced a notable number of natural 1s, possibly contributing to chaotic or failed actions during key moments.
- Some characters, like **Molly**, **Caduceus** and **Yasha**, have fewer extremes simply due to appearing in fewer episodes.

Overall, the contrast between Nat 1s and Nat 20s gives us a sense of how each character's luck shaped their journey through the campaign, sometimes heroic, sometimes hilariously tragic.
***

### Total Damage Dealt by Character

This chart shows the total amount of damage dealt by each character during Campaign 2. It gives us a sense of who took the most offensive actions in combat, highlighting key damage-dealers in the party.

Note that some characters may have lower totals due to fewer appearances or a more supportive playstyle. This chart reflects raw output, not necessarily impact.


In [55]:
# Filter to main characters only
main_characters = ["Beau", "Fjord", "Jester", "Veth", "Caleb", "Yasha", "Caduceus", "Molly", "Kingsley"]
filtered_df = df[df["Character"].isin(main_characters)]

# Group by character and sum total damage
damage_data = filtered_df.groupby("Character")["Damage"].sum().reset_index()
damage_data = damage_data.sort_values("Damage", ascending=False)

# Create bar chart
fig = px.bar(
    damage_data,
    x="Character",
    y="Damage",
    title="Total Damage Dealt by Character",
    text="Damage",
    color="Damage",
    color_continuous_scale="Reds"
)

fig.update_layout(
    xaxis_title="Character",
    yaxis_title="Total Damage",
    height=500
)

fig.update_traces(textposition="outside")

fig.show()

#### Insight: Who Did the Most Damage?

This chart clearly crowns **Beau** as the campaign's heavy hitter, dealing the highest total damage out of all the Mighty Nein. Her role as a frontline fighter and agile striker is reflected in her consistent combat presence and output. Which makes sense from the amount of rolls she did on the campagin.

Characters like **Fjord** and **Veth** also contributed significantly to the party’s overall damage, especially in combat-heavy arcs. Meanwhile, characters such as **Caduceus** and **Jester** took on more supportive roles, focusing on healing, spellcasting utility, or general chaos.

While damage doesn’t tell the full story of a character’s impact, it does spotlight the party’s combat MVPs, and Beau absolutely earned the title.
***

## Total Kills by Character

This chart shows the total number of confirmed kills made by each member of the Mighty Nein. While the number of rolls reflects activity, and damage reflects impact, the kill count reveals who landed the final blow most often.

Keep in mind that not all combat encounters resulted in a logged kill, and support roles or spellcasters may contribute significantly to combat without dealing the finishing blow.


In [56]:
# Filter to main characters
main_characters = ["Beau", "Fjord", "Jester", "Veth", "Caleb", "Yasha", "Caduceus", "Molly", "Kingsley"]
filtered_df = df[df["Character"].isin(main_characters)]

# Group by character and sum kills
kill_data = filtered_df.groupby("Character")["Kills"].sum().reset_index()
kill_data = kill_data.sort_values("Kills", ascending=False)

# Create bar chart
fig = px.bar(
    kill_data,
    x="Character",
    y="Kills",
    text="Kills",
    title="💀 Total Kills by Character",
    color="Kills",
    color_continuous_scale="Purples"
)

fig.update_layout(
    xaxis_title="Character",
    yaxis_title="Total Kills",
    height=500
)

fig.update_traces(textposition="outside")

fig.show()

#### Insight: Who Took the Most Lives?

Surprisingly, the top two characters in terms of confirmed kills are **Caleb** and **Veth**. While Beau was frequently on the frontlines, Caleb’s powerful area-of-effect spells and Veth’s lethal precision from range likely secured many final blows.

- **Caleb’s** high kill count aligns with his destructive spellcasting style, especially later in the campaign when he had access to powerful magic like Fireball and Wall of Fire.
- **Veth**, with her rogue skills and sharpshooter finesse, consistently delivered surgical strikes that finished off enemies.
- **Beau**, despite her high activity and damage, may have often softened enemies up but didn’t land the final blow as frequently.
- Support-focused characters like **Caduceus** and **Jester** have much lower kill counts—consistent with their roles in healing, crowd control, and chaotic antics.
***

### Roll Frequency Across Episodes

This line chart shows the number of dice rolls made in each episode of Campaign 2. Spikes often correlate with intense combat sequences or chaotic moments, while dips may indicate slower, roleplay-focused episodes.

Tracking the frequency of rolls over time helps us understand the campaign’s pacing and identify action-packed arcs.


In [57]:
from statsmodels.nonparametric.smoothers_lowess import lowess

# Group by episode
episode_rolls = df.groupby("Episode_Num").size().reset_index(name="Roll Count")
episode_rolls = episode_rolls.sort_values("Episode_Num")

# Generate smoothed trend
smoothed = lowess(episode_rolls["Roll Count"], episode_rolls["Episode_Num"], frac=0.15)

# Find top 3 spike episodes
top_episodes = episode_rolls.nlargest(3, "Roll Count")

# Create figure
fig = go.Figure()

# Main roll count line
fig.add_trace(go.Scatter(
    x=episode_rolls["Episode_Num"],
    y=episode_rolls["Roll Count"],
    mode="lines+markers",
    name="Roll Count",
    line=dict(color="royalblue", width=2),
    marker=dict(size=6)
))

# Smoothed trend line
fig.add_trace(go.Scatter(
    x=smoothed[:, 0],
    y=smoothed[:, 1],
    mode="lines",
    name="Trend Line",
    line=dict(color="firebrick", dash="dot", width=3)
))

# Highlight top 3 episodes
for _, row in top_episodes.iterrows():
    fig.add_annotation(
        x=row["Episode_Num"],
        y=row["Roll Count"],
        text=f"Ep {int(row['Episode_Num'])}",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

# Layout styling
fig.update_layout(
    title="📈 Roll Frequency per Episode (Enhanced)",
    xaxis_title="Episode Number",
    yaxis_title="Total Rolls",
    height=550,
    plot_bgcolor="#f8f8f8",
    hovermode="x unified"
)

fig.show()



#### Insight: Pacing and Chaos Over Time

This enhanced chart shows the number of dice rolls made in each episode of Campaign 2, along with a trend line to highlight broader pacing patterns.

- **Spikes** represent action-heavy sessions—often major battles or chaotic moments.
- **Dips** suggest quieter episodes focused on exploration, roleplay, or narrative development.
- The **trend line** gives us a smoothed view of the campaign's overall pacing, revealing arcs of increasing or decreasing intensity.
- **Highlighted points** mark the top three most roll-heavy episodes.
***

### Character Summary Statistics

This table presents a summary of key performance metrics for each character:
- Total rolls
- Average roll value
- Natural 1s
- Natural 20s
- Total damage dealt

It gives a concise overview of each character’s dice-based footprint across the campaign.


In [58]:
# Filter to main characters
main_characters = ["Beau", "Fjord", "Jester", "Veth", "Caleb", "Yasha", "Caduceus", "Molly", "Kingsley"]
filtered_df = df[df["Character"].isin(main_characters)]

# Group by character and compute summary stats
summary_table = filtered_df.groupby("Character").agg(
    Total_Rolls=("Total Value", "count"),
    Avg_Roll=("Total Value", "mean"),
    Nat_1s=("Is_Nat1", "sum"),
    Nat_20s=("Is_Nat20", "sum"),
    Total_Damage=("Damage", "sum")
).reset_index()

# Round the average roll for prettier display
summary_table["Avg_Roll"] = summary_table["Avg_Roll"].round(2)

# Sort by total rolls or whichever metric you like
summary_table = summary_table.sort_values("Total_Rolls", ascending=False)

summary_table

,Character,Total_Rolls,Avg_Roll,Nat_1s,Nat_20s,Total_Damage
0,Beau,2508,15.85,108,139,6585
6,Veth,1895,17.78,87,86,6007
3,Fjord,1726,16.14,65,109,5947
4,Jester,1660,15.57,83,71,4661
2,Caleb,1390,17.08,66,84,4376
7,Yasha,1140,17.41,56,65,5399
1,Caduceus,1079,16.59,28,49,3045
5,Molly,354,12.6,11,26,740


## Command to run streamlit app
python -m streamlit run CA2_dashboard.py